In [27]:
import os
import re

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import urlparse

from tqdm import tqdm


In [35]:
def checkURL(requested_url):
    if not urlparse(requested_url).scheme:
        requested_url = "https://" + requested_url
    return requested_url


def requestAndParse(requested_url):
    requested_url = checkURL(requested_url)
    try:
        # define headers to be provided for request authentication
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
        request_obj = Request(url=requested_url, headers=headers)
        page_html = urlopen(request_obj)
        page_soup = BeautifulSoup(page_html, "html.parser")
        return page_soup

    except Exception as e:
        print(e)

def crawlImageURL(soupObject):
    collection = []
    soup = soupObject
    for img in soup.find_all('img'):
        collection.append(img.attrs['src'])
    return set(collection)

In [38]:
soupObject = requestAndParse('https://www.pexels.com/vi-vn/tim-kiem/cat/')
imgCollection = crawlImageURL(soupObject)
len(imgCollection)

36